In [535]:
from scipy.special import binom as binomial
#compute probabilities of for given branch in the binary tree
def split_weights(n,k):
    v0=binomial(n-1,k)
    v1=binomial(n-1,k-1)
    return v0/(v0+v1),v1/(v0+v1)

#compute all probabilities for the binary tree
def circuit_layout(n,k):
    L=[]
    for i in range(n):
        for j in range(min(i,k-1),-1,-1):
            if n-i >= k-j:
                L.append([n-i,k-j,split_weights(n-i,k-j)])
    return L

# Define all the Helper Functions
(Everything as before)

In [536]:
def set_ancillas_to_num(c,ancillas,num):
    ind=0
    for i in reversed(bin(num)[2:]):
        if int(i)==1:
            c.add(gates.X(ancillas[ind]))
        ind+=1
    

In [537]:
def add_negates_for_check(c,ancillas,num):
    ind=0
    for i in reversed(bin(num)[2:]):
        if int(i)==0:
            c.add(gates.X(ancillas[ind]))
        ind+=1
    for i in range(len(bin(num)[2:]),len(ancillas)):
        c.add(gates.X(ancillas[i]))
    

In [538]:
def controlls_for_check_num(ancillas,num):
    l=[]
    c=0
    for i in reversed(bin(num)[2:]):
        if int(i)==1:
            l.append(ancillas[c])
        c+=1
    return l

In [539]:
def sub_one(c,ancillas,controlls):
    a=ancillas
    c.add(gates.X(a[0]).controlled_by(*controlls))
    for i in range(1,len(a)):
        controlls.append(a[i-1])
        c.add(gates.X(a[i]).controlled_by(*controlls))
        

In [540]:
def add_one(c,ancillas,controlls):
    a=ancillas
    for i in range(len(a)-1):
        controlls.append(a[i])

    #c.add(gates.X(a[0]).controlled_by(*controlls))
    for i in reversed(range(0,len(a))):
        c.add(gates.X(a[i]).controlled_by(*controlls))
        controlls.pop()

In [541]:
def row_add(c,col,row_res,row_addend,H,n,controlls):
    for i in range(col+1,n+1):
        g=gates.X(H[row_res,i])
        controlls.append(H[row_addend,i])
        c.add(g.controlled_by(*controlls))
        controlls.pop()
        
    

In [542]:
def add_pivot(c,identity,parity,H,n,r,controlls):
    i=identity
    p=parity
    c.add(gates.X(H[i,p]).controlled_by(*controlls))
    #loop for differnt rows in which the pivot is searched
    for l in range(r-(i+1)):
        #current row in which pivot is searched
        pr=i+l+1
        #need to add controls on permutation
        row_add(c,p,i,pr,H,n,[H[i,p]]+[H[i+1+o,p] for o in range(l+1)]+controlls)
        #add not for "if all preious were 0"-statement
        if l!=r-i-2:
            c.add(gates.X(H[pr,p]).controlled_by(*controlls))
    #revert all nots
    for l in range(r-i-1):
        c.add(gates.X(H[i+l,p]).controlled_by(*controlls))
    

In [543]:
def lower_col_elimination(c,identity,parity,H,n,r,controlls):
    i=identity
    p=parity
    #for each element below the current diagonal
    for l in range(r-i-1):
        #add row if needed to eliminate ones in column p
        row_add(c,p,i+l+1,i,H,n,[H[i+l+1,p]]+controlls)

In [544]:
def upper_col_elimination(c,identity,parity,H,n,r,controlls):
    i=identity
    p=parity
    #for each row above the current diagonal entry (identiy)
    for l in range(i):
        #eliminate ones by adding row[i] to that row
        #for Prange just do it on the last column (the syndrome)
        c.add(gates.X(H[l,n]).controlled_by(*([H[l,p],H[i,n]]+controlls)))
        #for Lee-Brickell do the row additions from column r=n-k-1 onwards
        #row_add(c,r-1,l,i,H,n,[H[l,p]]+controlls)

In [545]:
from math import log2,ceil
from qibo.models import Circuit
from qibo import gates
import numpy as np


In [546]:

def create_qibo_circuit(n,r):
    n_anc=int(ceil(log2(r+1)))
    c=Circuit(int(n+n_anc+r*(n+1)))
    L=circuit_layout(n,r)

    anc=[i for i in range(n,n+n_anc)]
    set_ancillas_to_num(c,anc,r)
    tmp=n

    for i in L:
        if tmp!=i[0]:
            sub_one(c,anc,[n-tmp])
            tmp=i[0]
        
        if(i[2]==(0,1)):
            add_negates_for_check(c,anc,i[1])
            c.add(gates.X(n-i[0]).controlled_by(*anc))
            add_negates_for_check(c,anc,i[1])    
        else:
            if i[0]!=n:
                add_negates_for_check(c,anc,i[1])
                c.add(gates.RY(n-i[0],float(2*np.arccos(np.sqrt(i[2][0])))).controlled_by(*anc))
                add_negates_for_check(c,anc,i[1])
            else:
                c.add(gates.RY(0,float(2*np.arccos(np.sqrt(i[2][0])))))
    sub_one(c,anc,[n-1])
    return c

### Generate test instance

In [547]:
def Reverse(tuples):
    new_tup = tuples[::-1]
    return new_tup

In [548]:
import itertools
r=3
n=5
solutions=[]

while(len(solutions)<2):
    solutions=[]
    A=np.random.randint(2, size=(r, n))
    while True:
        b=np.random.randint(2, size=(r,1))
        if not(all(b==0)):
            break
    comb=itertools.combinations([i for i in range(n)],r)
    Aprime=np.arange(r*r).reshape(r,r)
    for i in comb:
        ind=0
        for j in Reverse(i):
            for l in range(r):
                Aprime[l,ind]=A[l,j]
            ind+=1
        mask=0
        for j in i:
            mask^=(1<<(n-j-1))
        v=bin(mask)[2:]
        while (len(v)<n):
            v="0"+v
        try:
            sol=np.linalg.solve(Aprime,b)%2
            for it in reversed(sol):
                if it==0:
                    v="0"+v
                else:
                    v="1"+v
            solutions.append(v)
        except:
            pass

print("solve all possible linear systems, where" )
print("")
print("H =")
print(A)
print("")
print("b =")
print(b)
print("")
print("solutions =")
print(solutions)
#solve_gf2(A,b)

solve all possible linear systems, where

H =
[[0 1 0 1 0]
 [1 0 0 0 1]
 [1 0 1 1 0]]

b =
[[1]
 [1]
 [0]]

solutions =
['11111100', '10111010', '11011001', '10110110', '01110011', '10101101', '10101011', '11100111']


# New basic circuit design
1. create superposition
2. swap columns corrsponding to ones in the superposition to the front
3. gaussian on first ``r`` columns


In [549]:
def swap_cols(c,i,j,H,n,r,controlls):
    for row in range(r):
        c.add(gates.SWAP(H[row,i],H[row,j]).controlled_by(*controlls))
    
def swap_to_front(c,i,H,n,r,controlls):
    for col in reversed(range(i)):
        swap_cols(c,col+1,col,H,n,r,controlls)

In [550]:

perm=[i for i in range(n)]
n_anc=int(ceil(log2(r+1)))
ancillas=[i for i in range(n,n+n_anc)]

H=np.arange((n+1)*r).reshape(r,n+1)
c=n+n_anc

for j in range(n+1):
    for i in range(r):
        H[i,j]=c
        c+=1



In [551]:

def standard_gaussian_on_first_cols(c,n,r,H,A,b):
    #initialize matrix in quantum register
    for i in range(r):
        for j in range(n):
            if A[i, j] == 1:
                c.add(gates.X(H[i, j]))
    for i in range(r):
        if b[i] == 1:
            c.add(gates.X(H[i, n]))

    ##############################################
    #########        Swap Circuit        #########
    ##############################################

    for i in perm:
        if i!=0:
            swap_to_front(c,i,H,n,r,[perm[i]])

    ##############################################
    #########    Gaussian Elimination    #########
    ##############################################

    #add pivots and create upper triangular matrix
    for i in range(r-1):
        add_pivot(c,i,i,H,n,r,[])
        lower_col_elimination(c,i,i,H,n,r,[])

    #solve via back substitution
    for i in reversed(range(1,r)):
        #upper_col_elimination(c,0,0,H,n,r)
        upper_col_elimination(c,i,i,H,n,r,[])


#create superposition circuit
c=create_qibo_circuit(n,r)
#add Gaussian
standard_gaussian_on_first_cols(c,n,r,H,A,b)

#### TODO: weight check of last column####

#add measurements
c.add(gates.M(*([H[i,n] for i in range(r)]+perm)))




In [552]:
#test circuit
m=c(nshots=10000)
m.frequencies().keys()

if all(i in m.frequencies().keys() for i in solutions):
    print("Circuit works correct")
else:
    print("Circuit works erroneous")
m.frequencies()

Circuit works correct


Counter({'01101110': 929,
         '01110011': 1003,
         '10101011': 1006,
         '10101101': 1055,
         '10110101': 959,
         '10110110': 1032,
         '10111010': 1058,
         '11011001': 982,
         '11100111': 983,
         '11111100': 993})

## Lee-Brickell improvement

In [553]:
def find_set_bits(x,n):
    bits=[]
    for i in range(n):
        mask=1<<i
        if mask&x:
            bits.append(i)
    return bits

In [554]:
def col_add(c,col_res,col_addend,H,r,controlls):
    for i in range(r):
        g=gates.X(H[i,col_res])
        controlls.append(H[i,col_addend])
        c.add(g.controlled_by(*controlls))
        controlls.pop()
        
    

In [555]:

perm=[i for i in range(n)]
n_anc=int(ceil(log2(r+1)))
ancillas=[i for i in range(n,n+n_anc)]

H=np.arange((n+1)*r).reshape(r,n+1)
c=n+n_anc

for j in range(n+1):
    for i in range(r):
        H[i,j]=c
        c+=1

c=create_qibo_circuit(n,r)
standard_gaussian_on_first_cols(c,n,r,H,A,b)
enumerate_p_out_of_k(c,n,r,3,H)
c.add(gates.M(*([H[i,n] for i in range(r)]+perm)))

In [556]:
#test circuit
m=c(nshots=10000)
m.frequencies().keys()

if all(i in m.frequencies().keys() for i in solutions):
    print("Circuit works correct")
else:
    print("Circuit works erroneous")
m.frequencies()

Circuit works correct


Counter({'01101110': 1031,
         '01110011': 913,
         '10101011': 1042,
         '10101101': 938,
         '10110101': 982,
         '10110110': 1007,
         '10111010': 992,
         '11011001': 992,
         '11100111': 1065,
         '11111100': 1038})

In [557]:


#Assuming we computed the Gaussian elimination on the first n-k columns of H
#carrying out all necessary row additions also on the last k+1 columns of H,
#where the last column (with index n) holds the syndrome.
#This function now takes every size p subset of columns n-k to n-1 and adds it to column n (the syndrome)
#, checks the weight of the syndrome and if it is small enough sets the sign-flip-bit for grover.
def enumerate_p_out_of_k(c,n,r,p,H):
    #setb is a k bit binary number with exacly p bits set to one
    k=n-r
    setb = (1 << p) - 1;
    limit = (1 << k);
    
    while setb < limit:
    
        #get positions of set bits in setb
        columns_to_add=find_set_bits(setb,k)
        
        #add those columns to syndrome, note that indices need to be shifted by n-k
        for i in columns_to_add:
            col_add(c,n,i+r,H,r,[]) 
            
        ######## TODO: check weight of syndrome ###############
        #####  if weight is w-p set sign-flip-bit to one   ####
        #######################################################
        
        #reverse adding of columns to syndrome
        for i in columns_to_add:
            col_add(c,n,i+r,H,r,[]) 
        
        #gives next binary number with p bits out of k set to one
        c = setb & - setb;
        r = setb + c;
        setb = int(((r ^ setb) >> 2) / c) | int(r);


# New design for Saving (n-k)^2 qubits

### New instance generation
- for testing


In [726]:
import itertools
r=3
n=6
solutions=[]

while(len(solutions)<2):
    solutions=[]
    A=np.random.randint(2, size=(r, n))
    for i in range(r):
        for j in range(r):
            if i==j:
                A[j,n-r+i]=1
            else:
                A[j,n-r+i]=0
    while True:
        b=np.random.randint(2, size=(r,1))
        if not(all(b==0)):
            break
    comb=itertools.combinations([i for i in range(n)],r)
    Aprime=np.arange(r*r).reshape(r,r)
    for i in comb:
        ind=0
        c=0
        for t in i:
            if t>=r:
                break
            c+=1
        new_i=Reverse(i[:c])+i[c:]
        for j in new_i:
            for l in range(r):
                Aprime[l,ind]=A[l,j]
            ind+=1
        mask=0
        for j in i:
            mask^=(1<<(n-j-1))
        v=bin(mask)[2:]
        while (len(v)<n):
            v="0"+v
        try:
            
            sol=np.linalg.solve(Aprime,b)%2
            for it in reversed(sol):
                if it==0:
                    v="0"+v
                else:
                    v="1"+v
            solutions.append(v)
        except:
            pass

print("solve all possible linear systems, where" )
print("")
print("H =")
print(A)
print("")
print("b =")
print(b)
print("")
print("solutions =")
print(solutions)


solve all possible linear systems, where

H =
[[0 1 0 1 0 0]
 [1 1 0 0 1 0]
 [0 0 0 0 0 1]]

b =
[[1]
 [1]
 [1]]

solutions =
['101110001', '111100101', '101010101', '101010011', '111000111']


### Adaptation of the circuit itself

In [727]:
def swap_rows(c,i,j,H,n,r,controlls):
    for col in range(n-r+1):
        c.add(gates.SWAP(H[i,col],H[j,col]).controlled_by(*controlls))

In [728]:

def create_qibo_circuit(n,r):
    n_anc=int(ceil(log2(r+1)))
    c=Circuit(int(n+n_anc+r*(n-r+1)+n-r))
    L=circuit_layout(n,r)

    anc=[i for i in range(n,n+n_anc)]
    set_ancillas_to_num(c,anc,r)
    tmp=n

    for i in L:
        if tmp!=i[0]:
            sub_one(c,anc,[n-tmp])
            tmp=i[0]
        
        if(i[2]==(0,1)):
            add_negates_for_check(c,anc,i[1])
            c.add(gates.X(n-i[0]).controlled_by(*anc))
            add_negates_for_check(c,anc,i[1])    
        else:
            if i[0]!=n:
                add_negates_for_check(c,anc,i[1])
                c.add(gates.RY(n-i[0],float(2*np.arccos(np.sqrt(i[2][0])))).controlled_by(*anc))
                add_negates_for_check(c,anc,i[1])
            else:
                c.add(gates.RY(0,float(2*np.arccos(np.sqrt(i[2][0])))))
    sub_one(c,anc,[n-1])
    return c

In [729]:

def swap_bit_to_front(c,i,register,controlls):
    for col in reversed(range(i)):
        c.add(gates.SWAP(register[col+1],register[col]).controlled_by(*controlls))

In [730]:
perm=[i for i in range(n)]
n_anc=int(ceil(log2(r+1)))
n_extra=n-r
ancillas=[i for i in range(n,n+n_anc)]
extra=[i for i in range(n+n_anc,n+n_anc+n_extra)]

H=np.arange((n-r+1)*r).reshape(r,n-r+1)
c=n+n_anc+n_extra

for j in range(n-r+1):
    for i in range(r):
        H[i,j]=c
        c+=1

c=create_qibo_circuit(n,r)


In [732]:
#initialize registers with matrix / vector
for i in range(r):
### Only n-r columns now ###
    for j in range(n-r):
        ### last n-r columns of A need to be selected, so we have an offset of r now ###
        if A[i, j] == 1:
            c.add(gates.X(H[i, j]))
        ################
for i in range(r):
    if b[i] == 1:
        c.add(gates.X(H[i, n-r]))

set_ancillas_to_num(c,ancillas,r)

#do last lower idenity
#for every of the last r columns (corresponding to the idenity part of the parity check matrix in systematic form)
for j in reversed(range(r)):
    #for every position this column can take in the idenity
    for num in reversed(range(j+1,r+1)):
        #column j is (j+1)-th unit vector, so if j ones where in first part, the column is already finished, i.e., no swap needed
        if num != j+1:
            add_negates_for_check(c,ancillas,num)
            swap_rows(c,j ,num-1 ,H,n,r,ancillas+[perm[j+n-r]])
            add_negates_for_check(c,ancillas,num)
        #subtract one from the amount of remaining columns of the idenityt part to be identified
    sub_one(c,ancillas,[perm[j+n-r]])

#do front upper idenity
#swap columns corresponding to ones in the superposition among the first n-r columns to the front
for i in range(n-r):
    c.add(gates.X(extra[i]).controlled_by(perm[i]))
    swap_bit_to_front(c,i,extra,[perm[i]])
    swap_to_front(c,i,H,n-r,r,[perm[i]])
    #to bring ancillas back to zero
    sub_one(c,ancillas,[perm[i]])

#add pivots and create upper triangular matrix
for i in range(min(r,n-r)):
    add_pivot(c,i,i,H,n-r,r,[extra[i]])
    lower_col_elimination(c,i,i,H,n-r,r,[extra[i]])

#solve via back substitution
for i in reversed(range(1,min(r,n-r))):
    #upper_col_elimination(c,0,0,H,n,r)
    upper_col_elimination(c,i,i,H,n-r,r,[extra[i]])
############################################

    


c.add(gates.M(*([H[i,n-r] for i in range(r)]+perm)))



In [736]:
#test circuit
m=c(nshots=1000)
m.frequencies().keys()

if all(i in m.frequencies().keys() for i in solutions):
    print("Circuit works correct")
else:
    print("Circuit works erroneous")
    a=set([])
    for i in solutions:
        if i not in m.frequencies().keys() and i not in a:
            a.add(i)

    print("missing:")
    print(a)
    

Circuit works correct
